In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import os
# from dotenv import load_dotenv

# load_dotenv(os.path.expanduser("~/.nmdc_mongo.env"))

In [3]:
from dotenv import load_dotenv

load_dotenv("../../.env.localhost")

True

TODO

- data object `data_object_type` field with [enum](https://docs.google.com/document/d/1uq6J__NbCUezsD16VTVJa-3_HJn5DacASBmFw3xQ3Bc/edit) codes

In [4]:
from nmdc_runtime.pipelines.core import preset_normal_env
from nmdc_runtime.resources.mongo import get_mongo


mongo = get_mongo(preset_normal_env.run_config)
db = mongo.db

In [5]:
db.list_collection_names()

['biosample_set',
 'metaproteomics_analysis_activity_set',
 'data_object_set',
 'objects',
 'capabilities',
 'operations',
 'omics_processing_set',
 'date_created',
 'etl_software_version',
 'triggers',
 'nom_analysis_activity_set',
 'activity_set',
 'notes',
 'read_based_analysis_activity_set',
 'metabolomics_analysis_activity_set',
 'metagenome_assembly_set',
 'study_set',
 'ids',
 'mags_activity_set',
 'metagenome_annotation_activity_set',
 'nmdc_schema_version',
 'users',
 'read_QC_analysis_activity_set',
 'genome_feature_set',
 'sites',
 'raw.functional_annotation_set',
 'object_types',
 'functional_annotation_set',
 'workflows']

In [6]:
from nmdc_runtime.pipelines.core import preset_normal_env
from nmdc_runtime.resources.core import get_runtime_api_site_client


client = get_runtime_api_site_client(preset_normal_env.run_config)

In [7]:
gold_etl_latest = db.objects.find_one({"name": "nmdc_database.json.zip"}, sort=[("created_time", -1)])

ajrh-6sff-63


In [8]:
rv = client.get_object_bytes(gold_etl_latest["id"])

In [9]:
from io import BytesIO
import json
from zipfile import ZipFile

with ZipFile(BytesIO(rv.content)) as myzip:
    # may be e.g. 'metadata-translation/src/bin/output/nmdc_database.json' rather than 'nmdc_database.json'
    name = next(n for n in myzip.namelist() if n.endswith("nmdc_database.json"))
    with myzip.open(name) as f:
        nmdc_database = json.load(f)

In [10]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate

In [12]:
nmdc_db_collection_names = set(nmdc_jsonschema["definitions"]["Database"]["properties"])
def reset_database(db):
    for coll_name in nmdc_db_collection_names:
        db.drop_collection(coll_name)
        db[coll_name].create_index("id", unique=True)
        
reset_database(db)

# fix biosample.part_of and add docs

In [13]:
from pprint import pprint

from toolz import assoc_in, dissoc, get_in

from nmdc_runtime.api.core.util import pick

new_docs = []
for doc in nmdc_database["biosample_set"]:
    if "part of" in doc:
        doc = assoc_in(doc, ["part_of"], get_in(["part of"], doc))
        doc = dissoc(doc, "part of")
    new_docs.append(doc)

nmdc_database["biosample_set"] = new_docs

In [14]:
ok = nmdc_jsonschema_validate(nmdc_database)

In [15]:
mongo.add_docs(nmdc_database)

@dehays:
> FYI - the problem I introduced was to make has_input required on omics_processing - that lead to all the EMSL omics_processing getting dropped because the association with samples doesn’t happen there until after ETL

# GOLD IDs to IGSNs

In [34]:
from dagster import build_solid_context

from nmdc_runtime.solids.core import local_file_to_api_object as lftao

context = build_solid_context(resources={"mongo": mongo, "runtime_api_site_client": client})


def local_file_to_api_object(file_info):
    return lftao(context, file_info)

#obj = local_file_to_api_object({"storage_path": storage_path, "mime_type": 'text/csv'})

<ipython-input-34-d41a4b73840e>:5: ExperimentalWarning: "build_solid_context" is an experimental function. It may break in future versions, even between dot releases. To mute warnings for experimental functionality, invoke warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning) or use one of the other methods described at https://docs.python.org/3/library/warnings.html#describing-warning-filters.
  context = build_solid_context(resources={"mongo": mongo, "runtime_api_site_client": client})


In [16]:
oid__gold_ids_to_igns = "c6hs-4rzh-31"
doc = db.objects.find_one({"id": oid__gold_ids_to_igns})
assert doc["name"] == "2020-23-12-brodie-Gs0135149-soil-metadata.csv"

In [17]:
from collections import defaultdict
import csv
from io import StringIO
import re

GOLD_ID_IDX = 5
IGSN_IDX = 2

igsn_golds = defaultdict(list)

gold_id_pattern = re.compile(r"Gb\d+")

f = StringIO(client.get_object_bytes(oid__gold_ids_to_igns).text)
reader = csv.reader(f)
for row in reader:
    gold_id = row[GOLD_ID_IDX]
    igsn = row[IGSN_IDX]
    if gold_id_pattern.fullmatch(gold_id):
        igsn_golds[igsn].append(gold_id)

In [18]:
from pprint import pprint

from toolz import get_in

new_biosample_docs = []

for igsn, golds in igsn_golds.items():
    igsn_curie = "igsn:"+igsn
    doc = db.biosample_set.find_one({"id": {"$in": [f"gold:{g}" for g in golds]}})
    doc = assoc_in(doc, ["id"], igsn_curie)
    doc = dissoc(doc, "_id")
    new_biosample_docs.append(doc)

assert len(new_biosample_docs) == len(igsn_golds)

In [19]:
mongo.add_docs({"biosample_set": new_biosample_docs})

In [20]:
from pymongo import DeleteMany
from toolz import concat

requests = [DeleteMany({"id": {"$in": ["gold:"+g for g in concat(igsn_golds.values())]}})]
rv = mongo.db.biosample_set.bulk_write(requests)
rv.deleted_count

48

# Update omics_processing_set references to biosample_set ids

In [21]:
goldid_igsn = {}
for igsn, gids in igsn_golds.items():
    for gid in gids:
        goldid_igsn[gid] = igsn

In [22]:
requests = []
to_replace = {"gold:"+k: "igsn:"+v for k, v in goldid_igsn.items()}

for doc in db.omics_processing_set.find({"has_input": {"$in": list(to_replace)}}):
    operations = {"$set": {
        "has_input": [to_replace.get(i, i) for i in doc["has_input"]],
    }}
    requests.append({"filter": {"_id": doc["_id"]}, "update": operations})

In [23]:
from pymongo import UpdateOne

rv = db.omics_processing_set.bulk_write([UpdateOne(**r) for r in requests])

In [24]:
rv.modified_count

93

In [25]:
docs = [dissoc(doc, "_id") for doc in db.omics_processing_set.find()]

In [26]:
ok = nmdc_jsonschema_validate({"omics_processing_set": docs})

# Update omics_processing_set references from EMSL ids to IGSNs

Skip this -- it updates zero documents!

In [27]:
EMSL_IDS_IDX = 7
IGSN_IDX = 2

igsn_emsls = {}

emsl_ids_pattern = re.compile(r"\d+")

f = StringIO(client.get_object_bytes(oid__gold_ids_to_igns).text)
reader = csv.reader(f)
for row in reader:
    emsl_ids = row[EMSL_IDS_IDX]
    igsn = row[IGSN_IDX]
    ids = emsl_ids_pattern.findall(emsl_ids)
    # XXX some rows have emsl ids but no IGSN, so igsn.strip() check here
    if igsn.strip() and ids:
        igsn_emsls[igsn] = ids

In [28]:
emslid_igsn = {}
for igsn, eids in igsn_emsls.items():
    for eid in eids:
        emslid_igsn[eid] = igsn

In [29]:
n_with_emsl_id = db.omics_processing_set.count_documents(
    {"id": {"$in": ["emsl:"+i for i in emslid_igsn]}})

In [30]:
requests = []
to_replace = {"emsl:"+k: "igsn:"+v for k, v in emslid_igsn.items()}
to_replace.update({"emsl:output_"+k: "igsn:"+v for k, v in emslid_igsn.items()})

def omit(blacklist, d):
    return keyfilter(lambda k: k not in blacklist, d)

def sans_mongo_id(d):
    return omit(["_id"], d)


for doc in db.omics_processing_set.find({"has_input": {"$in": list(to_replace)}}):
    operations = {"$set": {
        "has_input": [to_replace.get(i, i) for i in doc["has_input"]],
    }}
    requests.append({"filter": {"_id": doc["_id"]}, "update": operations})

In [31]:
if requests:
    rv = db.omics_processing_set.bulk_write([UpdateOne(**r) for r in requests])
    print(rv.modified_count)

# metaP_stegen.ipynb

In [92]:
to_fetch = [{
    # >200MB
    "url": "https://portal.nersc.gov/cfs/m3408/meta/stegen_MetaProteomicAnalysis_activity.json",
    "type": "metaproteomics_analysis_activity_set",
}, {
    # ~50KB
    "url": "https://portal.nersc.gov/cfs/m3408/meta/stegen_emsl_analysis_data_objects.json",
    "type": "data_object_set"
}]

In [149]:
import re

pattern = re.compile(r"https?://(?P<domain>[^/]+)/(?P<path>.+)")

def url_to_name(url):
    m = pattern.match(url)
    return f"{'.'.join(reversed(m.group('domain').split('.')))}__{m.group('path').replace('/', '.')}"

In [150]:
from pathlib import Path

import requests

def download_them_all(to_fetch):
    for i, spec in enumerate(to_fetch):
        url = spec["url"]
        name = url_to_name(url)
        print(f"{i+1}/{len(to_fetch)}: fetching {url}")
        rv = requests.get(url)
        print(f"saving as {name}")
        with open(f'/Users/dwinston/Downloads/{name}', 'w') as f:
            json.dump(rv.json(), f)

In [68]:
download_them_all(to_fetch)

1/2: fetching stegen_MetaProteomicAnalysis_activity.json
2/2: fetching stegen_emsl_analysis_data_objects.json


In [69]:
from toolz import identity, dissoc, assoc_in

metaP_field_map = {
    "PeptideSequence": ("peptide_sequence", identity),
    "sum(MASICAbundance)": ("peptide_sum_masic_abundance", int),
    "SpectralCount": ("peptide_spectral_count", int),
    "BestProtein": ("best_protein", identity),
    "min(QValue)": ("min_q_value", float),
    
    "peptide_sequence": ("peptide_sequence", identity),
    "peptide_sum_masic_abundance": ("peptide_sum_masic_abundance", int),
    "peptide_spectral_count": ("peptide_spectral_count", int),
    "best_protein": ("best_protein", identity),
    "min_q_value": ("min_q_value", float),
}


def map_fields(doc, field_map=None):
    for k_old, todo in field_map.items():
        if k_old in doc:
            k_new, fn = todo
            v_new = fn(doc[k_old])
            doc = dissoc(doc, k_old)
            doc = assoc_in(doc, [k_new], v_new)
    return doc


def correct_metaP_doc(doc):
    if not "has_peptide_quantifications" in doc:
        return doc
    new_items = [
        map_fields(item, metaP_field_map) for item in doc["has_peptide_quantifications"]
    ]
    doc = assoc_in(
        doc,
        ["has_peptide_quantifications"],
        new_items,
    )
    return doc

In [70]:
def fetch_downloaded_json(name):
    with open(f'/Users/dwinston/Downloads/{name}') as f:
        return json.load(f)

In [93]:
from collections import defaultdict

def fetch_metaP_validate_and_add(to_fetch):
    to_add = defaultdict(list)
    for i, spec in enumerate(to_fetch):
        url = spec["url"]
        name = Path(url).name
        collection_name = spec["type"]
        print(f"{i+1}/{len(to_fetch)}: fetching {name} ({collection_name})")
        docs = fetch_downloaded_json(name)
        if not isinstance(docs, list):
            docs = [docs]
        docs = [correct_metaP_doc(d) for d in docs]
        to_add[collection_name].extend(docs)
    print("validating")
    nmdc_jsonschema_validate(to_add)
    print("adding")
    mongo.add_docs(to_add, validate=False)

In [94]:
fetch_metaP_validate_and_add(to_fetch)

1/2: fetching stegen_MetaProteomicAnalysis_activity.json (metaproteomics_analysis_activity_set)
2/2: fetching stegen_emsl_analysis_data_objects.json (data_object_set)
validating
adding


In [95]:
to_fetch = [{
    "url": "https://portal.nersc.gov/project/m3408/meta/501128_1781_100340_stegen_MetaProteomicAnalysis_activity.json",
    "type": "metaproteomics_analysis_activity_set",
}, {
    "url": "https://portal.nersc.gov/project/m3408/meta/501128_1781_100340_stegen_emsl_analysis_data_objects.json",
    "type": "data_object_set"
}]

In [96]:
download_them_all(to_fetch)

1/2: fetching 501128_1781_100340_stegen_MetaProteomicAnalysis_activity.json
2/2: fetching 501128_1781_100340_stegen_emsl_analysis_data_objects.json


In [97]:
fetch_metaP_validate_and_add(to_fetch)

1/2: fetching 501128_1781_100340_stegen_MetaProteomicAnalysis_activity.json (metaproteomics_analysis_activity_set)
2/2: fetching 501128_1781_100340_stegen_emsl_analysis_data_objects.json (data_object_set)
validating
adding


# mongo_etl_demo.ipynb

In [99]:
to_fetch = [{
    "url": "https://portal.nersc.gov/cfs/m3408/meta/mt_annotation_objects.json",
    "type": "metagenome_annotation_activity_set"
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/mt_annotation_data_objects.json",
    "type": "data_object_set"
}, {
    "url": "https://portal.nersc.gov/project/m3408/meta/metagenomeAssembly_activity.json",
    "type": "metagenome_assembly_set",
}, {
    "url": "https://portal.nersc.gov/project/m3408/meta/metagenomeAssembly_data_objects.json",
    "type": "data_object_set",
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/ReadbasedAnalysis_activity.json",
    "type": "read_based_analysis_activity_set"
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/ReadbasedAnalysis_data_objects.json",
    "type": "data_object_set"
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/MAGs_activity.json",
    "type": "mags_activity_set",
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/MAGs_data_objects.json",
    "type": "data_object_set"
}, {
    "url": "https://portal.nersc.gov/project/m3408/meta/readQC_activity.json",
    "type": "read_QC_analysis_activity_set"
}, {
    "url": "https://portal.nersc.gov/project/m3408/meta/readQC_activity_data_objects.json",
    "type": "data_object_set"
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/img_mg_annotation_objects.json",
    "type": "metagenome_annotation_activity_set",
}, {
    "url": "https://portal.nersc.gov/cfs/m3408/meta/img_mg_annotation_data_objects.json",
    "type": "data_object_set",
}, {
    "url": "https://nmdcdemo.emsl.pnnl.gov/metabolomics/registration/gcms_metabolomics_data_products.json",
    "type": "data_object_set"
}, {
    "url": "https://nmdcdemo.emsl.pnnl.gov/nom/registration/ftms_nom_data_products.json",
    "type": "data_object_set"
}]

In [100]:
download_them_all(to_fetch)

1/14: fetching mt_annotation_objects.json
2/14: fetching mt_annotation_data_objects.json
3/14: fetching metagenomeAssembly_activity.json
4/14: fetching metagenomeAssembly_data_objects.json
5/14: fetching ReadbasedAnalysis_activity.json
6/14: fetching ReadbasedAnalysis_data_objects.json
7/14: fetching MAGs_activity.json
8/14: fetching MAGs_data_objects.json
9/14: fetching readQC_activity.json
10/14: fetching readQC_activity_data_objects.json
11/14: fetching img_mg_annotation_objects.json
12/14: fetching img_mg_annotation_data_objects.json
13/14: fetching gcms_metabolomics_data_products.json
14/14: fetching ftms_nom_data_products.json


In [102]:
from collections import defaultdict

def fetch_validate_and_add(to_fetch):
    to_add = defaultdict(list)
    for i, spec in enumerate(to_fetch):
        url = spec["url"]
        name = Path(url).name
        collection_name = spec["type"]
        print(f"{i+1}/{len(to_fetch)}: fetching {name} ({collection_name})")
        docs = fetch_downloaded_json(name)
        if not isinstance(docs, list):
            docs = [docs]
        to_add[collection_name].extend(docs)
    print("validating")
    nmdc_jsonschema_validate(to_add)
    print("adding")
    mongo.add_docs(to_add, validate=False)

In [103]:
fetch_validate_and_add(to_fetch)

1/14: fetching mt_annotation_objects.json (metagenome_annotation_activity_set)
2/14: fetching mt_annotation_data_objects.json (data_object_set)
3/14: fetching metagenomeAssembly_activity.json (metagenome_assembly_set)
4/14: fetching metagenomeAssembly_data_objects.json (data_object_set)
5/14: fetching ReadbasedAnalysis_activity.json (read_based_analysis_activity_set)
6/14: fetching ReadbasedAnalysis_data_objects.json (data_object_set)
7/14: fetching MAGs_activity.json (mags_activity_set)
8/14: fetching MAGs_data_objects.json (data_object_set)
9/14: fetching readQC_activity.json (read_QC_analysis_activity_set)
10/14: fetching readQC_activity_data_objects.json (data_object_set)
11/14: fetching img_mg_annotation_objects.json (metagenome_annotation_activity_set)
12/14: fetching img_mg_annotation_data_objects.json (data_object_set)
13/14: fetching gcms_metabolomics_data_products.json (data_object_set)
14/14: fetching ftms_nom_data_products.json (data_object_set)
validating
adding


In [145]:
manifests = [{
    "url": (
        "https://nmdcdemo.emsl.pnnl.gov/metabolomics/registration/"
        "gcms_metabolomics_metadata_products.json"
    ),
    "type": "metabolomics_analysis_activity_set"
}, {
    "url": (
        "https://nmdcdemo.emsl.pnnl.gov/nom/registration/"
        "ftms_nom_metadata_products.json"
    ),
    "type": "nom_analysis_activity_set"
}]

In [151]:
import requests


to_fetch = []

for m in manifests:
    urls = requests.get(m["url"]).json()
    for url in urls:
        to_fetch.append({"url": url, "type": m["type"]})

In [152]:
to_fetch

[{'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_S2_0_10_7_Metab.json',
  'type': 'metabolomics_analysis_activity_set'},
 {'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_S3_20_30_14_Metab.json',
  'type': 'metabolomics_analysis_activity_set'},
 {'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_S3_10_20_5_Metab.json',
  'type': 'metabolomics_analysis_activity_set'},
 {'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_S3_50_60_4_Metab.json',
  'type': 'metabolomics_analysis_activity_set'},
 {'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_S1_40_50_17_Metab.json',
  'type': 'metabolomics_analysis_activity_set'},
 {'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_N1_0_10_13_Metab.json',
  'type': 'metabolomics_analysis_activity_set'},
 {'url': 'https://nmdcdemo.emsl.pnnl.gov/metabolomics/metadata/Froze_Core_2015_S1_30_40

In [158]:
import concurrent.futures
import json

import requests
from tqdm.notebook import tqdm

def fetch_json(url):
    return requests.get(url).json()


def download_them_all_parallel(to_fetch):
    error_urls = []
    pbar = tqdm(total=len(to_fetch))
    urls = [spec["url"] for spec in to_fetch]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_url = {
            executor.submit(fetch_json, url): url
            for url in urls
        }
        for future in concurrent.futures.as_completed(future_to_url):
            pbar.update(1)
            url = future_to_url[future]
            try:
                payload = future.result()
            except Exception as e:
                error_urls.append((url, str(e)))
            else:
                name = url_to_name(url)
                with open(f'/Users/dwinston/Downloads/{name}', 'w') as f:
                    json.dump(payload, f)

    pbar.close()
    return error_urls

In [159]:
download_them_all_parallel(to_fetch)

  0%|          | 0/997 [00:00<?, ?it/s]

[]

In [160]:
def fetch_downloaded_json_given_url(url):
    name = url_to_name(url)
    with open(f'/Users/dwinston/Downloads/{name}') as f:
        return json.load(f)

In [170]:
import requests
from tqdm.notebook import tqdm

def validate_and_add_parallel(to_fetch):
    nmdc_db = defaultdict(list)
    error_urls = []
    pbar = tqdm(total=len(to_fetch))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_spec = {
            executor.submit(fetch_downloaded_json_given_url, spec["url"]): spec
            for spec in to_fetch
        }
        for future in concurrent.futures.as_completed(future_to_spec):
            spec = future_to_spec[future]
            try:
                payload = future.result()
            except Exception as e:
                error_urls.append((spec["url"], url_to_name(spec["url"]), str(e)))
            else:
                nmdc_db[spec["type"]].append(payload)
            pbar.update(1)

    pbar.close()
    return nmdc_db, error_urls

In [171]:
nmdc_db, error_urls = validate_and_add_parallel(to_fetch)

  0%|          | 0/997 [00:00<?, ?it/s]

In [175]:
ok = nmdc_jsonschema_validate(nmdc_db)

In [176]:
mongo.add_docs(nmdc_db)

In [179]:
# TODO load functional_annotation_set.find({has_function: {$regex: '^KEGG'}})

# ghissue_252_253_linked_samples.ipynb

# ghissue_255.ipynb

# ghissue_272.ipynb

# ensure_biosample_set_study_id.ipynb

# Biosample linking update

In [ ]:
rows = []
with open("../src/data/2021-02-03-stegen_biosample_linking_update.csv") as f:
    next(f) # skip header row
    for row in f:
        line = row.strip()
        tokens = line.split(",")
        if tokens[-1] == '':
            rows.append(tokens[:-1])
        else:
            rows.append(tokens)

In [ ]:
import json
from pprint import pprint


with open("../src/data/2021-02-04-stegen_biosample_template.json") as f:
    s = f.read()

try:
    stegen_sample_template = json.loads(s)
except json.JSONDecodeError:
    stegen_sample_template = json.loads(
        s.replace('\n', '')\
        .replace("$BIOSAMPLE_ID", '"$BIOSAMPLE_ID"')\
        .replace("“", '"')
    )

In [ ]:
import re

gold_pattern = re.compile(r"Gb\d+")

def prefix_sample_id(s):
    if ":" in s:
        return s
    elif re.fullmatch(gold_pattern, s):
        return "gold:" + s
    else:
        return "emsl:" + s

In [ ]:
omics = []
for i, row in enumerate(rows):
    omics.append({
        "omics_id": row[0],
        "omics_type": row[1],
        "sample_name": row[2],
        "sample_id": prefix_sample_id(row[3]),
        "new": len(row) > 4 and row[4] == "TRUE"
    })

In [ ]:
existing_ids = [
    d["id"] for d in
    db.biosample_set.find({"id": {"$in": [o["sample_id"] for o in omics]}}, ["id"])
]

In [ ]:
from toolz import assoc_in, get_in

def transform_in(doc, keys, fn):
    initial = get_in(keys, doc)
    transformed = fn(initial)
    return assoc_in(doc, keys, transformed)

In [ ]:
def fill_template(template, sample_id, sample_name):
    doc = assoc_in(template, ["id"], sample_id)
    doc = transform_in(
        doc, ["identifier", "has_raw_value"],
        lambda s: s.replace("$BIOSAMPLE_NAME", sample_name)
    )
    doc = transform_in(
        doc, ["name"],
        lambda s: s.replace("$BIOSAMPLE_NAME", sample_name)
    )
    return doc

In [ ]:
def term_subdocs_to_id_strings(doc):
    keys_with_term_ids = [
        k for k in doc
        if isinstance(doc[k], dict)
        and "term" in doc[k]
        and "id" in doc[k]["term"]
    ]
    for k in keys_with_term_ids:
        doc = assoc_in(doc, [k, "term"], doc[k]["term"]["id"])
    return doc

In [ ]:
new_samples = {}
for o in omics:
    if o["new"]:
        new_samples[o["sample_id"]] = o["sample_name"]

docs = []

for sample_id, sample_name in new_samples.items():
    doc = fill_template(stegen_sample_template, sample_id, sample_name)
    doc = term_subdocs_to_id_strings(doc)
    docs.append(doc)

In [ ]:
from toolz import get_in, assoc_in

def un_raw_value(doc, key):
    value = get_in([key, "has_raw_value"], doc)
    if value is not None:
        return assoc_in(doc, [key], value)
    else:
        return doc

raws = [
    "ecosystem",
    "collection_date",
    "community",
    "ecosystem_category",
    "ecosystem_subtype",
    "ecosystem_type",
    "geo_loc_name",
    "habitat",
    "identifier",
    "location",
    "ncbi_taxonomy_name",
    "sample_collection_site",
    "specific_ecosystem",
]
for key in raws:
    docs = [un_raw_value(d, key) for d in docs]

In [ ]:
docs

In [ ]:
from pymongo import ReplaceOne

rv = db.biosample_set.bulk_write([ReplaceOne({"id": doc["id"]}, doc, upsert=True) for doc in docs])

In [ ]:
rv.upserted_count

In [ ]:
omics = [
    transform_in(o, ["omics_id"], lambda s: "emsl:"+s if ":" not in s else s)
    for o in omics
]

In [ ]:
omics_ids = [o["omics_id"] for o in omics]

found_omics_ids = [
    d["id"] for d in
    db.omics_processing_set.find({"id": {"$in": omics_ids}},["id"])
]

In [ ]:
assert set(omics_ids) == set(found_omics_ids)

In [ ]:
omics_updates = {}
for o in omics:
    omics_updates[o["omics_id"]] = o

In [ ]:
from toolz import dissoc

replacing_omics_type = {}

requests = []

for doc in db.omics_processing_set.find({"id": {"$in": omics_ids}}):
    omics_type = get_in(["omics_type"], doc)
    updates = omics_updates[doc["id"]]
    if omics_type != updates["omics_type"]:
        replacing_omics_type[doc["id"]] = {"from": omics_type, "to": updates["omics_type"]}
    doc = assoc_in(doc, ["omics_type"], updates["omics_type"])
    doc = assoc_in(doc, ["has_input"], [updates["sample_id"]])
    requests.append(ReplaceOne({"id": doc["id"]}, dissoc(doc, "_id")))

In [ ]:
replacing_omics_type

In [ ]:
len(requests)

In [ ]:
rv = db.omics_processing_set.bulk_write(requests)

In [ ]:
rv.modified_count

In [ ]:
rows = []
with open("../src/data/2021-02-04-Brodie_biosample_linking_update - Sheet1.csv") as f:
    next(f) # skip header row
    for row in f:
        line = row.strip()
        tokens = line.split(",")
        if tokens[-1] == '':
            rows.append(tokens[:-1])
        else:
            rows.append(tokens)

In [ ]:
omics = []
for i, row in enumerate(rows):
    omics.append({
        "omics_id": "emsl:" + row[0].strip(),
        "omics_type": row[1].strip(),
        "sample_name": row[2].strip(),
        "sample_id": "igsn:" + row[3].strip(),
    })

In [ ]:
omics_ids = [o["omics_id"] for o in omics]

found_omics_ids = [
    d["id"] for d in
    db.omics_processing_set.find({"id": {"$in": omics_ids}},["id"])
]

In [ ]:
assert set(omics_ids) == set(found_omics_ids)

In [ ]:
omics_updates = {}
for o in omics:
    omics_updates[o["omics_id"]] = o

In [ ]:
replacing_omics_type = {}

requests = []

for doc in db.omics_processing_set.find({"id": {"$in": omics_ids}}):
    omics_type = get_in(["omics_type"], doc)
    updates = omics_updates[doc["id"]]
    if omics_type != updates["omics_type"]:
        replacing_omics_type[doc["id"]] = {"from": omics_type, "to": updates["omics_type"]}
    doc = assoc_in(doc, ["omics_type"], updates["omics_type"])
    doc = assoc_in(doc, ["has_input"], [updates["sample_id"]])
    requests.append(ReplaceOne({"id": doc["id"]}, dissoc(doc, "_id")))

In [ ]:
replacing_omics_type

In [ ]:
len(requests)

In [ ]:
rv = db.omics_processing_set.bulk_write(requests)

In [ ]:
rv.modified_count

Remove the 40 Wrighton EMSL omics_processing and data object documents that relate to isolates

In [ ]:
mfilter = {
    "part_of": ["gold:Gs0114675"],
    "processing_institution": "Environmental Molecular Sciences Lab"
}

db.omics_processing_set.count_documents(filter=mfilter)

In [ ]:
from pymongo import DeleteMany
from toolz import concat

requests = []


docs = list(db.omics_processing_set.find(mfilter, ["id", "has_output"]))
omics_processing_ids = [d["id"] for d in docs]
data_object_ids = list(concat(d["has_output"] for d in docs))

assert len(omics_processing_ids) == db.data_object_set.count_documents({"id": {"$in": data_object_ids}})

In [ ]:
rv1 = db.omics_processing_set.delete_many({"id": {"$in": omics_processing_ids}})
rv2 = db.data_object_set.delete_many({"id": {"$in": data_object_ids}})

In [ ]:
rv1.deleted_count, rv2.deleted_count

add 5 Brodie samples used at EMSL but not JGI

In [ ]:
biosamples = [
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000I",
      "identifier":"igsn:IEWFS000I"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000K",
      "identifier":"igsn:IEWFS000K"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000B",
      "identifier":"igsn:IEWFS000B"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000A",
      "identifier":"igsn:IEWFS000A"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000J",
      "identifier":"igsn:IEWFS000J"
   }
]

In [ ]:
add_to_db(biosamples, db, collection_name="biosample_set")

In [ ]:
assert db.biosample_set.count_documents({"id": {"$in": [d["id"] for d in biosamples]}}) == len(biosamples)